In [1]:
import numpy as np
import xtensor_climate_fun as xt

from IPython.display import display

# Load `xtensor::snyder_edd`

In [2]:
print(xt.snyder_edd.__doc__)

snyder_edd(arg0: numpy.ndarray[float], arg1: numpy.ndarray[float], arg2: float) -> numpy.ndarray[float]

Return the snyder EDD



In [3]:
def py_snyder_edd(tasmin, tasmax, threshold):

    # compute useful quantities for use in the transformation
    snyder_mean = ((tasmax + tasmin)/2)
    snyder_width = ((tasmax - tasmin)/2)
    snyder_theta = np.arcsin( (threshold - snyder_mean)/snyder_width )

    # the trasnformation is computed using numpy arrays, taking advantage of
    # numpy's second where clause. Note that in the current dev build of
    # xarray, xr.where allows this functionality. As soon as this goes live,
    # this block can be replaced with xarray
    res = np.where(
        tasmin < threshold,
        np.where(
            tasmax > threshold,
            ((snyder_mean - threshold) * (np.pi/2 - snyder_theta)
                + (snyder_width * np.cos(snyder_theta))) / np.pi,
            0),
        snyder_mean - threshold)

    return res

In [4]:
a = np.arange(15).reshape(3, 5) + 5
b = a + 10
edd_xt = xt.snyder_edd(a, b, 10)
edd_py = py_snyder_edd(a, b, 10)
display(a)
display(b)
display(edd_xt)
display(edd_py)

//anaconda/envs/xtensor/lib/python3.5/site-packages/ipykernel/__main__.py:6: RuntimeWarning: invalid value encountered in arcsin


array([[ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19]])

array([[15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24],
       [25, 26, 27, 28, 29]])

array([[  1.59154943,   2.12348782,   2.72065891,   3.38773784,   4.1355986 ],
       [  5.        ,   6.        ,   7.        ,   8.        ,   9.        ],
       [ 10.        ,  11.        ,  12.        ,  13.        ,  14.        ]])

array([[  1.59154943,   2.12348782,   2.72065891,   3.38773784,   4.1355986 ],
       [  5.        ,   6.        ,   7.        ,   8.        ,   9.        ],
       [ 10.        ,  11.        ,  12.        ,  13.        ,  14.        ]])

# Performance test python vs xtensor

In [5]:
tasmin = np.random.random((1440, 720, 10))*15+273.15
tasmax = tasmin + 10

In [6]:
% timeit py_snyder_edd(tasmin, tasmax, 303.15)

//anaconda/envs/xtensor/lib/python3.5/site-packages/ipykernel/__main__.py:6: RuntimeWarning: invalid value encountered in arcsin


804 ms ± 40.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
% timeit xt.snyder_edd(tasmin, tasmax, 303.15)

491 ms ± 6.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
